In [4]:
!pip install --upgrade transformers
!pip install simpletransformers
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

Requirement already up-to-date: transformers in /usr/local/lib/python3.6/dist-packages (4.2.2)
     |████████████████████████████████| 102kB 4.3MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.4-cp36-none-any.whl size=74842 sha256=f7ae49578fc540f2442545ece5fccea24b7871a013a31d6f6e770c0bd4f2c278
  Stored in directory: /root/.cache/pip/wheels/9e/11/04/5160b8815b0cc7cf574bdc6d053e510169ec264c8791b4ec3a
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=08b5cf41def258345b97e9e35c3e6dd4ae7449377281ac10cc23c8ccf8336169
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built watchdog pathtools
  Found existing installation: watchdog 1.0.2
    Uninstalling watchdog-1.0.2:
      Successfully uninstalled watchdog-1.0.2


In [5]:
import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 12.7 GB  |     Proc size: 296.9 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total     16280MB


In [6]:
import numpy as np
import pandas as pd
from google.colab import files
from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')
import gc
from scipy.special import softmax
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
import sklearn
from sklearn.metrics import log_loss
from sklearn.metrics import *
from sklearn.model_selection import *
import re
import random
import torch
pd.options.display.max_colwidth = 200

def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

seed_all(2)

In [7]:
train = pd.read_csv("train.csv")
train.head()

,Unnamed: 0,id,label,tweet
0,0,1,0,fingerprint Pregnancy Test android apps beautiful cute health igers iphoneonly iphonesia iphone
1,1,2,0,Finally a transparant silicon case Thanks to my uncle yay Sony Xperia S sonyexperias
2,2,3,0,We love this Would you go talk makememories unplug relax iphone smartphone wifi connect
3,3,4,0,Im wired I know Im George I was made that way iphone cute daventry home
4,4,5,1,What amazing service Apple wont even talk to me about a question I have unless I pay them 19 95 for their stupid support


In [8]:
train["label"].value_counts()

0    5894
1    2026
Name: label, dtype: int64

In [9]:
test = pd.read_csv("test.csv")
test.head()

,Unnamed: 0,id,tweet
0,0,7921,I hate the new iphone upgrade Wont let me download apps ugh apple sucks
1,1,7922,currently shitting my fucking pants apple iMac cashmoney raddest swagswagswag
2,2,7923,Id like to puts some CD ROMS on my iPad is that possible Yes but wouldnt that block the screen
3,3,7924,My ipod is officially dead I lost all my pictures and videos from the 1D and 5sos concert and from Vet Camp hatinglife sobbing
4,4,7925,Been fighting iTunes all night I only want the music I paid for


In [10]:
train1=train.drop(['Unnamed: 0', 'id'],axis=1)
test1=test.drop(['Unnamed: 0', 'id'],axis=1)
test1['label']=0

In [11]:
train1.head()

,label,tweet
0,0,fingerprint Pregnancy Test android apps beautiful cute health igers iphoneonly iphonesia iphone
1,0,Finally a transparant silicon case Thanks to my uncle yay Sony Xperia S sonyexperias
2,0,We love this Would you go talk makememories unplug relax iphone smartphone wifi connect
3,0,Im wired I know Im George I was made that way iphone cute daventry home
4,1,What amazing service Apple wont even talk to me about a question I have unless I pay them 19 95 for their stupid support


In [12]:
test1.head()

,tweet,label
0,I hate the new iphone upgrade Wont let me download apps ugh apple sucks,0
1,currently shitting my fucking pants apple iMac cashmoney raddest swagswagswag,0
2,Id like to puts some CD ROMS on my iPad is that possible Yes but wouldnt that block the screen,0
3,My ipod is officially dead I lost all my pictures and videos from the 1D and 5sos concert and from Vet Camp hatinglife sobbing,0
4,Been fighting iTunes all night I only want the music I paid for,0


In [13]:
neworder = ['tweet', 'label']
train1=train1.reindex(columns=neworder)
train1.head()

,tweet,label
0,fingerprint Pregnancy Test android apps beautiful cute health igers iphoneonly iphonesia iphone,0
1,Finally a transparant silicon case Thanks to my uncle yay Sony Xperia S sonyexperias,0
2,We love this Would you go talk makememories unplug relax iphone smartphone wifi connect,0
3,Im wired I know Im George I was made that way iphone cute daventry home,0
4,What amazing service Apple wont even talk to me about a question I have unless I pay them 19 95 for their stupid support,1


In [14]:
test1.head()

,tweet,label
0,I hate the new iphone upgrade Wont let me download apps ugh apple sucks,0
1,currently shitting my fucking pants apple iMac cashmoney raddest swagswagswag,0
2,Id like to puts some CD ROMS on my iPad is that possible Yes but wouldnt that block the screen,0
3,My ipod is officially dead I lost all my pictures and videos from the 1D and 5sos concert and from Vet Camp hatinglife sobbing,0
4,Been fighting iTunes all night I only want the music I paid for,0


In [15]:
%%time
err=[]
y_pred_tot=[]

fold=StratifiedKFold(n_splits=20, shuffle=True, random_state=2)
i=1
for train_index, test_index in fold.split(train1,train1['label']):
    train1_trn, train1_val = train1.iloc[train_index], train1.iloc[test_index]
    model = ClassificationModel('roberta', 'roberta-large', use_cuda=True,num_labels=2, args={'train_batch_size':32,
                                                                         'reprocess_input_data': True,
                                                                         'overwrite_output_dir': True,
                                                                         'fp16': False,
                                                                         'do_lower_case': False,
                                                                         'num_train_epochs': 2,
                                                                         'max_seq_length': 64,
                                                                         'regression': False,
                                                                         'manual_seed': 2,
                                                                         "learning_rate":3e-5,
                                                                         'weight_decay':0,
                                                                         "save_eval_checkpoints": False,
                                                                         "save_model_every_epoch": False,
                                                                         "silent": True})
    model.train_model(train1_trn)
    raw_outputs_val = model.eval_model(train1_val)[1]
    raw_outputs_val = softmax(raw_outputs_val,axis=1)[:,1]
    print(f"Log_Loss: {log_loss(train1_val['label'], raw_outputs_val)}")
    err.append(log_loss(train1_val['label'], raw_outputs_val))
    raw_outputs_test = model.eval_model(test1)[1]
    raw_outputs_test = softmax(raw_outputs_test,axis=1)[:,1]
    y_pred_tot.append(raw_outputs_test)
print("Mean LogLoss: ",np.mean(err))
final=pd.DataFrame()
final['id']=test['id']
final['label']=np.mean(y_pred_tot, 0)
print(final.shape)
final.to_csv('20fold_rbl_2_3e5_32_64_0.csv',index=False)

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.o

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Log_Loss: 0.17295919353169728


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.o

Log_Loss: 0.23779249585158316


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.o

Log_Loss: 0.23388288265715326
Mean LogLoss:  0.21410143597624876
(1953, 2)
CPU times: user 1h 25min 23s, sys: 39min 50s, total: 2h 5min 14s
Wall time: 2h 9min 53s


In [16]:
sub= pd.read_csv("sub.csv")
sub.head()

,id,label
0,7921,0.915689
1,7922,0.550219
2,7923,0.830658
3,7924,0.816336
4,7925,0.888799


In [19]:
sub['label'] = sub.iloc[:,1:].mean(axis=1)

In [20]:
sub['label']

0       0.915689
1       0.550219
2       0.830658
3       0.816336
4       0.888799
          ...   
1948    0.782788
1949    0.003245
1950    0.856330
1951    0.576683
1952    0.002492
Name: label, Length: 1953, dtype: float64

In [22]:
sub['label'] = (sub.label >= 0.5).astype(int)

In [23]:
sub['label']

0       1
1       1
2       1
3       1
4       1
       ..
1948    1
1949    0
1950    1
1951    1
1952    0
Name: label, Length: 1953, dtype: int64

In [24]:
sub['label'].value_counts()

0    1358
1     595
Name: label, dtype: int64

In [25]:
sub.to_csv('roberta.csv', index=False)